In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

In [2]:
df = pd.read_csv('Data/ListaCobroDetalle2025.csv')
print (df.shape)
df.head()


(2114172, 9)


,idListaCobro,idCredito,consecutivoCobro,idBanco,montoExigible,montoCobrar,montoCobrado,fechaCobroBanco,idRespuestaBanco
0,155938,738973,41396434,2,622.87,622.87,0.00,NaN,4.0
1,155938,739017,41396435,2,1069.11,1069.11,0.00,NaN,4.0
2,155939,739185,41396436,2,4340.83,4340.83,4340.83,02/01/2025,0.0
3,155940,732324,41396437,2,2134.21,2134.21,0.00,NaN,4.0
4,155940,737028,41396438,2,815.76,815.76,0.00,NaN,4.0


In [3]:
# Copia la clase para poder hacerle una limpia
df2 = df.copy()

In [4]:
# Sortvalues by idCredito
df2 = df2.sort_values(by=['idCredito'])
if 'fechaCobroBanco' in df2.columns:
    df2 = df2.drop(columns=['fechaCobroBanco'])
df2 = df2.fillna(0)

In [5]:
# Agregamos un orden
df2['orden'] = df2.groupby('idCredito').cumcount()


In [6]:
# Procesar bancos
top4_bancos = df2['idBanco'].value_counts().nlargest(4).index
bancos = {12:0, 2:1, 72:2, 21:3}
bancos.update({x: 4 for x in df2['idBanco'].unique() if x not in bancos})
df2['idBanco'] = df2['idBanco'].apply(lambda x:x if x in top4_bancos else 4)
# Dummies
df2 = pd.get_dummies(df2, columns=['idBanco'], prefix='banco')


In [7]:
# vemos las columnas
print(df2.columns)

Index(['idListaCobro', 'idCredito', 'consecutivoCobro', 'montoExigible',
       'montoCobrar', 'montoCobrado', 'idRespuestaBanco', 'orden', 'banco_2',
       'banco_4', 'banco_12', 'banco_21', 'banco_72'],
      dtype='object')


In [8]:
# Agrupamos los creditos
df2 = df2.drop(columns=['idListaCobro', 'consecutivoCobro'])
# Extraemos los creditos para luego indexar despues
creditos = df2['idCredito'].unique()
Cobradores = df2.groupby('idCredito')

In [9]:
print (Cobradores.size())

idCredito
9872      18
10983      3
30466     18
31375      2
33591      3
          ..
755683     1
755695     1
755726     1
755859    16
756595     5
Length: 28602, dtype: int64


In [10]:
# Creamos el historial del grupo 
historial_por_grupo = []
for _, grupo in Cobradores:
    historial_por_grupo.append(grupo.drop(columns=['idCredito']).values.tolist())
print (len(historial_por_grupo))

28602


In [11]:
# Sacamos el modelo
import torch
from models.transaction_classifier import Transaction
input_dim = len(historial_por_grupo[0][0])
input_history = len(historial_por_grupo[0][0])
model = Transaction(input_dim, input_history)
model.load_state_dict(torch.load('model_epoch_0_step_2000.pth'))

<All keys matched successfully>

In [12]:
# Convertimos a vectores los valores
from readData import convertir_a_tensor
import numpy as np
import random

labels = [random.randint(0, 2) for _ in range(len(historial_por_grupo))]
print(len(labels))
train_loader = convertir_a_tensor(historial_por_grupo, labels)

28602


In [13]:
print (len(list(train_loader)))

28602


In [14]:
from tqdm import tqdm

In [19]:
lista_predicciones = []
for i,(historical, actual, label) in tqdm(enumerate(train_loader)):
    # Enviamos a la GPU
    historical = historical.unsqueeze(0)
    actual = actual.unsqueeze(0)
    # Predecimos
    pred = model(historical, actual)
    # Convertimos a numpy
    pred = pred.cpu().detach().numpy()
    # Lo volvemos un valor binario
    pred = np.where(pred > 0.5, 1, 0)
    # Lo agregamos a la lista
    lista_predicciones.append(pred)

# Restart the tqdm


0it [00:00, ?it/s]


In [20]:
# Convertimos a un dataframe
df_predicciones = pd.DataFrame(lista_predicciones, columns=['prediccion'])

In [21]:
print (df_predicciones.head())
# Vemos el tamaño
print (df_predicciones.shape)

Empty DataFrame
Columns: [prediccion]
Index: []
(0, 1)


In [18]:
# Vemos el tama;o de los creditos
creditos = df.groupby('idCredito').size()
print (creditos)


idCredito
9872      18
10983      3
30466     18
31375      2
33591      3
          ..
755683     1
755695     1
755726     1
755859    16
756595     5
Length: 28602, dtype: int64
